In [2]:
import pandas as pd
import numpy as np
import math
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import grangercausalitytests
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
import statistics
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

file_name = "/content/drive/My Drive/Subjects/"
file_name1 = '/content/drive/My Drive/Subjects/Ambient/Ambient/'

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_name + str(k) + '/' + str(k) + '_votes' + '.csv'

'/content/drive/My Drive/Subjects/63/63_votes.csv'

In [ ]:
k=76
Ambient = pd.read_csv(file_name + str(k) + '/' + str(k) + '_votes.csv')
Ambient = Ambient["Actual Temperature"]
Ambient = pd.DataFrame(Ambient)
#Ambient["Actual Temperature"][0] = Ambient["Actual Temperature"][1]
#Ambient = Ambient.drop(0)
#Ambient = pd.read_csv(file_name + str(k) + '/' + str(k) + '_votes' + '.csv',sep = '')
#Ambient = Ambient.drop(["Timestamp for sample frequency every 1 min","Relative_Humidity"], axis = 1)

In [ ]:
k=35
Ambient = pd.read_csv(file_name + str(k) + '/Sensor 2-starts-2019-07-17-10-29-00-ends-2019-07-17-12-32-58.csv',sep = ',')
Ambient = Ambient.drop(["Timestamp for sample frequency every 1 min","Relative_Humidity"], axis = 1)

In [ ]:
Ambient["time"] = Ambient.index
Ambient["time"] *= 240

In [ ]:
Ambient

,Actual Temperature,time
0,19.9,0
1,20.2,240
2,21.3,480
3,22.7,720
4,23.8,960
5,23.9,1200
6,24.6,1440
7,25.3,1680
8,26.6,1920
9,27.2,2160


In [ ]:
Temp = pd.DataFrame()
Temp["time"] = np.arange(0, Ambient["time"].max(), 2)

In [ ]:
Temp = Temp.merge(Ambient, how = 'outer').interpolate()

In [ ]:
Temp.to_csv(str(k)+ "_Env_1.txt", index=False)

In [ ]:
Ambient

,time,Temperature_Celsius
0,0,22.284000
1,2,22.284713
2,4,22.285427
3,6,22.286140
4,8,22.286853
...,...,...
2126,4252,26.036360
2127,4254,26.036720
2128,4256,26.037080
2129,4258,26.037440


In [ ]:
Ambient = pd.read_csv(file_name1 + str(83) + '_Env_1.txt',sep = ' ',header = None)

In [ ]:
Ambient.head()

,0,1,2,3,4
0,2020-03-05,13:27:43,NaN,20.8,502.0
3,2020-03-05,13:27:47,30.7,20.7,495.0
4,2020-03-05,13:27:49,30.5,20.7,501.0
7,2020-03-05,13:27:53,31.2,20.7,503.0
8,2020-03-05,13:27:55,31.2,20.7,496.0


In [ ]:
Ambient = Ambient[ Ambient[3] < 30] 

In [ ]:
Ambient = Ambient[Ambient[4].isna() == False]

In [8]:
Ambient.to_csv(str(83)+ "_Env_1.txt", sep = ' ', index=False)

In [5]:
Ambient = pd.read_csv(file_name1 + str(83) + '_Env_1.txt',sep = ',',header = None)

In [7]:
Ambient.to_csv(str(83)+ "_Env_1.txt", index=False)

In [ ]:
Ambient = pd.read_csv(file_name1 + str(83) + '_Env_1.txt',sep = '',header = None)
Ambient.columns = ("Day","Hour","Humidity","Ambient_Temp","unknown")
Ambient['Hour']  = pd.to_datetime(Ambient['Hour'])
Ambient['time'] = Ambient['Hour'] - Ambient.iloc[0,1]
Ambient['time'] = Ambient['time'].dt.total_seconds().astype(int)

Ambient = Ambient.dropna()
Ambient = Ambient.reset_index()
Ambient = Ambient.drop(columns = ["unknown","index","Day","Hour","Humidity"])
Ambient = Ambient.drop(Ambient["time"][Ambient["time"] < 90].index)
Ambient = Ambient.reset_index()
Ambient = Ambient.drop(columns = ["index"])


In [ ]:
Ambient

,Ambient_Temp,time
0,20.0,91
1,20.0,93
2,20.0,95
3,20.0,97
4,20.0,99
...,...,...
2389,22.7,4889
2390,22.7,4891
2391,22.6,4893
2392,22.7,4895


In [ ]:
#def merge(k):
k=35
#### reading datasets
HR = pd.read_csv(file_name + str(k) + '/HR.csv',header = None)
HR_median = pd.read_csv(file_name + str(k) + '/HR_median.csv',header = None)
EDA = pd.read_csv(file_name + str(k) + '/EDA_envelope.csv', header = None)
TEMP = pd.read_csv(file_name + str(k) + '/TEMP.csv', header = None)
BVP = pd.read_csv(file_name + str(k) + '/BVP.csv', header = None)
BVP_envelope = pd.read_csv(file_name + str(k) + '/BVP_envelope.csv', header = None)
BVP_median = pd.read_csv(file_name + str(k) + '/BVP_median.csv', header = None)
Ambient = pd.read_csv(file_name1 + str(k) + '_Env_1.txt',sep = ' ',header = None)

#### removing first two rows that show start timestamp and frequency
HR_starttime = int(HR.iloc[0])
HR_fq = HR.iloc[1]
HR = HR.rename(columns = {0 : "HR"})

EDA_starttime = int(TEMP.iloc[0])
EDA_fq = 4
EDA = EDA.rename(columns = {0 : "EDA"})

TEMP_starttime = int(TEMP.iloc[0])
TEMP_fq = TEMP.iloc[1]
TEMP = TEMP.rename(columns = {0 : "TEMP"})

BVP_starttime = int(BVP.iloc[0])
BVP_fq = BVP.iloc[1]
BVP = BVP.rename(columns = {0 : "BVP"})

MAX = max(HR_starttime,EDA_starttime,BVP_starttime,TEMP_starttime)
HR_dif = MAX - HR_starttime
EDA_dif = MAX - EDA_starttime
BVP_dif = MAX - BVP_starttime
TEMP_dif = MAX - TEMP_starttime

if HR_dif > 0:
    HR = HR.drop(range(2,int(HR_dif*HR_fq)))
    HR_median = HR_median.drop(range(2,int(HR_dif*HR_fq)))
if EDA_dif > 0:
    EDA = EDA.drop(range(2,int(EDA_dif*EDA_fq)))
if BVP_dif > 0 :
    BVP = BVP.drop(range(2,int(BVP_dif * BVP_fq)))
    BVP_median = BVP_median.drop(range(2,int(BVP_dif * BVP_fq)))
    BVP_envelope = BVP_envelope.drop(range(2,int(BVP_dif * BVP_fq)))
if TEMP_dif > 0 :
    TEMP = TEMP.drop(range(2,int(TEMP_dif*TEMP_fq)))

#### removing first 3 mintutes of the experience for each subject
EDA = EDA.reset_index()
EDA = EDA.drop(range(0,int(180 * EDA_fq)))
EDA = pd.DataFrame(EDA)
EDA = EDA.drop(columns = "index")

HR = HR.reset_index()
HR = HR.drop(range(0,int(180 * HR_fq)))
HR = pd.DataFrame(HR)
HR = HR.drop(columns = "index")

HR_median = HR_median.reset_index()
HR_median = HR_median.drop(range(0,int(180 * HR_fq)))
HR_median = pd.DataFrame(HR_median)
HR_median = HR_median.drop(columns = "index")

TEMP = TEMP.reset_index()
TEMP = TEMP.drop(range(0,int(180 * TEMP_fq)))
TEMP = pd.DataFrame(TEMP)
TEMP= TEMP.drop(columns = "index")

BVP = BVP.reset_index()
BVP = BVP.drop(range(0,int(180 * BVP_fq)))
BVP = pd.DataFrame(BVP)
BVP= BVP.drop(columns = "index")

BVP_envelope = BVP_envelope.reset_index()
BVP_envelope = BVP_envelope.drop(range(0,int(180 * BVP_fq)))
BVP_envelope = pd.DataFrame(BVP_envelope)
BVP_envelope= BVP_envelope.drop(columns = "index")

BVP_median = BVP_median.reset_index()
BVP_median = BVP_median.drop(range(0,int(180 * BVP_fq)))
BVP_median = pd.DataFrame(BVP_median)
BVP_median= BVP_median.drop(columns = "index")


if k == 25:
  Ambient = pd.read_csv(file_name1 + str(k) + '_Env_1.txt',sep = ',')
else:
  Ambient.columns = ("Day","Hour","Humidity","Ambient_Temp","unknown")
  Ambient['Hour']  = pd.to_datetime(Ambient['Hour'])
  Ambient['time'] = Ambient['Hour'] - Ambient.iloc[0,1]
  Ambient['time'] = Ambient['time'].dt.total_seconds().astype(int)
  
Ambient = Ambient.dropna()
Ambient = Ambient.reset_index()
Ambient = Ambient.drop(columns = ["unknown","index","Day","Hour","Humidity"])
Ambient = Ambient.drop(Ambient["time"][Ambient["time"] < 90].index)
Ambient = Ambient.reset_index()
Ambient = Ambient.drop(columns = ["index"])

#### only focus on the ascending trend of the ambient temperature
Upper_head = Ambient["time"][Ambient["Ambient_Temp"] == Ambient["Ambient_Temp"].max()].max()

Ambient = Ambient.drop(range(math.floor(Upper_head/2),Ambient.shape[0]))
Ambient = Ambient.reset_index()
Ambient = Ambient.drop(columns = ["index"])

EDA = EDA.reset_index()
EDA = EDA.drop(range(int(Upper_head*EDA_fq),EDA.shape[0]))
EDA = pd.DataFrame(EDA)
EDA = EDA.drop(columns = "index")

HR = HR.reset_index()
HR = HR.drop(range(int(Upper_head*HR_fq),HR.shape[0]))
HR = pd.DataFrame(HR)
HR = HR.drop(columns = "index")

HR_median = HR_median.reset_index()
HR_median = HR_median.drop(range(int(Upper_head*HR_fq),HR_median.shape[0]))
HR_median  = pd.DataFrame(HR_median)
HR_median = HR_median.drop(columns = "index")

TEMP = TEMP.reset_index()
TEMP = TEMP.drop(range(int(Upper_head*TEMP_fq),TEMP.shape[0]))
TEMP = pd.DataFrame(TEMP)
TEMP = TEMP.drop(columns = "index")

BVP = BVP.reset_index()
BVP = BVP.drop(range(int(Upper_head*BVP_fq),BVP.shape[0]))
BVP = pd.DataFrame(BVP)
BVP= BVP.drop(columns = "index")

BVP_envelope = BVP_envelope.reset_index()
BVP_envelope = BVP_envelope.drop(range(int(Upper_head*BVP_fq),BVP_envelope.shape[0]))
BVP_envelope = pd.DataFrame(BVP_envelope)
BVP_envelope = BVP_envelope.drop(columns = "index")

BVP_median = BVP_median.reset_index()
BVP_median = BVP_median.drop(range(int(Upper_head*BVP_fq),BVP_median.shape[0]))
BVP_median = pd.DataFrame(BVP_median)
BVP_median = BVP_median.drop(columns = "index")

#### assigning time to each feature
HR['time'] = list(np.arange(0, int(Upper_head * HR_fq),float(1/HR_fq)))
HR_median['time'] = list(np.arange(0, int(Upper_head * HR_fq),float(1/HR_fq)))
HR_median.columns = ["HR_median", "time"]
EDA['time'] = list(np.arange(0, int(Upper_head),float(1/EDA_fq)))
TEMP['time'] = list(np.arange(0, int(Upper_head),float(1/TEMP_fq)))
BVP['time'] = list(np.arange(0, int(Upper_head),float(1/64)))
BVP_envelope['time'] = list(np.arange(0, int(Upper_head),float(1/64)))
BVP_median['time'] = list(np.arange(0, int(Upper_head),float(1/64)))

#### preparing BVPs
temp = pd.DataFrame(BVP["BVP"].groupby(np.arange(len(BVP["BVP"]))//1024).std())
time = pd.DataFrame(BVP["time"].groupby(np.arange(len(BVP["time"]))//1024).median().round())
BVP = pd.concat([time,temp],axis = 1)

BVP_envelope.columns = ["BVP_envelope", "time"]
time = pd.DataFrame(list(np.arange(0, int(Upper_head),float(1/4))))
BVP_envelope = BVP_envelope["BVP_envelope"].groupby(np.arange(len(BVP_envelope["BVP_envelope"]))//16).mean()
BVP_envelope = pd.concat([time,BVP_envelope],axis = 1)
BVP_envelope.columns = ["time", "BVP_envelope"]

BVP_median.columns = ["BVP_median", "time"]
BVP_median = pd.DataFrame(BVP_median)
BVP_median = BVP_median["BVP_median"].groupby(np.arange(len(BVP_median["BVP_median"]))//16).mean()
BVP_median = pd.concat([time, BVP_median], axis=1)
BVP_median.columns = ["time","BVP_median"]


#merging all datasess
Dataset = EDA.merge(HR,on = "time", how='outer')
Dataset = Dataset.merge(HR_median,on = "time", how='outer')
Dataset['HR'] = Dataset['HR'].interpolate()
Dataset['HR_median'] = Dataset['HR_median'].interpolate()

Dataset = Dataset.merge(TEMP, on = "time", how = 'outer')

Dataset = Dataset.merge(BVP, on = "time", how = 'outer')
Dataset['BVP'] = Dataset['BVP'].interpolate()
Dataset = Dataset.merge(BVP_envelope, on = "time", how = 'outer')
Dataset = Dataset.merge(BVP_median, on = "time", how = 'outer')

Ambient['time'] = list(np.arange(0,Upper_head-1,2))
Dataset = Dataset.merge(Ambient, on = "time", how = 'outer')
Dataset['Ambient_Temp'] = Dataset['Ambient_Temp'].astype(float)
Dataset['Ambient_Temp'] = Dataset['Ambient_Temp'].interpolate()

TV = pd.read_csv(file_name + str(k) + '/' + str(k) + '_votes&ambient.csv')
TV["time"] = TV["Unnamed: 0"]*2
TV = TV.drop(["Time", "Unnamed: 0"],axis=1)
Dataset = Dataset.merge(TV, on = "time", how = 'outer')
Dataset['Thermal Preference'] = Dataset['Thermal Preference'].interpolate()
#Dataset["Thermal Preference"][(Dataset["Thermal Preference"] <= 1) & (Dataset["Thermal Preference"] >= -1)] = 0
#Dataset["Thermal Preference"][(Dataset["Thermal Preference"] > 1) | (Dataset["Thermal Preference"] < -1)] = 1
#Dataset["Thermal Preference"][Dataset["Thermal Preference"] > 1] = 1
#Dataset["Thermal Preference"][Dataset["Thermal Preference"] < -1] = -1
Dataset = Dataset.dropna()

# Changing to minute based!
Dataset = Dataset.groupby(np.arange(len(Dataset["BVP_envelope"]))//240).mean()

FileNotFoundError: ignored

In [ ]:
Dataset

,EDA,time,HR,HR_median,TEMP,BVP,BVP_envelope,BVP_median,Ambient_Temp,Thermal Preference
0,0.316819,37.875,75.374563,76.103573,31.212250,32.622473,404.774057,6.999358,17.421875,2.000000
1,0.282043,97.875,75.744813,75.584708,31.352833,30.419752,377.347648,4.578714,17.413333,2.000000
2,0.270425,157.875,74.987250,74.877115,31.280000,29.557256,352.318357,2.428729,17.423333,2.000000
3,0.280898,217.875,74.243542,74.079292,31.156250,29.818733,329.705698,4.503767,17.426667,2.000000
4,0.312396,277.875,73.125438,73.827375,31.156750,24.678701,309.529315,2.579724,17.633958,2.000000
5,0.363852,337.875,69.346375,73.740000,31.215083,40.286922,291.808833,1.558701,18.127292,1.952083
6,0.434199,397.875,73.002250,73.708562,31.345500,42.654319,276.563826,5.728848,18.572500,1.000000
7,0.522370,457.875,74.210604,73.698938,31.570250,47.836149,263.813836,1.454082,18.992500,1.000000
8,0.627298,517.875,75.520458,74.022792,31.681833,34.692602,253.578536,2.055607,19.367708,1.000000
9,0.747917,577.875,75.057604,74.349063,31.708500,37.342574,245.877513,4.002203,19.675833,0.952083
